In [ ]:
#getting training and testing data  
import numpy as np
import pandas as pd

train_data = pd.read_csv("netflix/TrainingRatings.txt", names=["MovieID", "UserID", "Rating"])
test_data = pd.read_csv("netflix/TestingRatings.txt", names=["MovieID", "UserID", "Rating"])

In [ ]:
#creating matrix comparing movieIDs directly to userIDs. 
user_itemTrain = train_data.pivot_table(index="UserID", columns="MovieID", values="Rating")

#make matrix for userIDs and their respective Vi mean
train_means = user_itemTrain.mean(axis=1)

#calculating vij - mean matrix
vijmeans = user_itemTrain.sub(train_means, axis = 0)

In [ ]:
#perform the Pearson correlation function using numpy rather than pandas because its like 10000% faster for some reason??? Computers are magic!
trainT = user_itemTrain.T
trainT = trainT.fillna(0)
weights = pd.DataFrame(np.corrcoef(trainT.values, rowvar=False), index = trainT.columns, columns = trainT.columns)

In [ ]:
#making predictions based on test data

predicted_vote = []

for index, row in test_data.iterrows():
    #get movie and user
    j = row[0]
    a = row[1]
    #check if user is actually in training data
    if a in train_means.index:
        va = train_means.loc[a]
    #if not, then set it to 3 (presumably the average of a 1-5 star scale)
    else:
        va = 3
    vijmeansforj = vijmeans.loc[:,j]
    wai = weights.loc[a,:].copy()
    sumwai = wai.abs().sum()

    if sumwai != 0:
        rightside = (wai *vijmeansforj).sum()/sumwai
    else:
        rightside = 0

    prediction = va + rightside
    predicted_vote.append(round(prediction))

In [ ]:
#calculating RMSE and MAE
RMSE = (((test_data['Rating'] - predicted_vote)**2).mean())**0.5
MAE = (abs(test_data['Rating'] - predicted_vote)).mean()
print("RMSE: " + str(round(RMSE, 6)))
print("MAE:  " + str(round(MAE, 6)))